<a href="https://colab.research.google.com/github/OnroerendErfgoed/scriptorium/blob/main/notebooks/03_error_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Where did we go wrong?

As in life, things will go wrong sooner or later. Luckily REST services are built to handle errors gracefully. The main tool it uses for this is the HTTP Response Code. Every response we get from an HTTP server includes a status code. This status code tells us whether the request was succesfull or not and possibly why. All response codes are triple digit numbers. The first digit tells you in which broad category you are:

* *200-299*: These response codes indicate success. By far the most common code in this category and probably the most common one of all is *200 OK*, signaling that all's well.
* *300-399*: These response codes indicate redirection, telling the client the answer to their question can be found elsewhere.
* *400-499*: These response codes indicate a client error, something the client might be able to fix be changing the request. Common errors in this category are:
 * *400 Bad Request*: Your request was unclear.
 * *404 Not Found*: The resource you're looking for does not exist.
 * *401 Unauthorized*: I can't send you the resource because I don't know who you are. Please authenticate first.
 * *403 Forbidden*: I can't send you the resource because I know who you are and you're not allowed to see the resource. Get more permissions.
* *500-599*: These response codes indicate a server error. Something went wrong in the server and the client generally has not control over it. The most common error in this category is *500 Internal Server Error*.

While there are a lot more error codes, you will do fine with the ones listed here. Remember, if it's starts with a 4, you should change. If it starts with a 5, the server is have issues and you can only wait a bit or contact the server administrator.

For our examples, we'll use a different dataset. This time we'll use the (Beeldbank)[https://beeldbank.onroerenderfgoed.be]. Keep in mind that we're querying for the metadata about images here, not the actual images.

#Does this resource exist?
First, we'll go back to our [basic script](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/01_get_resource.ipynb) that got a single resource at a time.

In [ ]:
import requests
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/afbeeldingen/409414',
    headers = {'Accept': 'application/json'}
)

# Turn the response's JSON data into a Python dictionary
data = response.json()

# Print the response's JSON data
print(json.dumps(data, sort_keys=True, indent=4))


If we change the URI to an invalid URI (eg. by changing the `409414` to `a`), we'll trigger an error. And our script will terminate. We can handle this in a much cleaner way, allowing our script to keep running, even when there's an error.

In [ ]:
import requests
import json

# Make a request and store the response
response = requests.get(
    'https://id.erfgoed.net/afbeeldingen/409414',
    headers = {'Accept': 'application/json'}
)

# Check to see if the response was ok (HTTP code between 200 and 400)
if response.ok:
  # Turn the response's JSON data into a Python dictionary
  data = response.json()

  # Print the response's JSON data
  print(json.dumps(data, sort_keys=True, indent=4))
else:
  print(f"Something went wrong: {response.status_code} - {response.text}")

Another way of handling errors, is by throwing Exceptions. This basically tells the code to stop doing what it's doing and start asking for help. It's not that usefull in a small, standalone script, but it works very well when you're building a library and you're working with functions that are being called by other functions that are being called by ... This kind of error handling gives you a stacktrace, a path through all the code the Exception was thrown from.

In [ ]:
import requests
import json

def get_url(url):
  # Make a request and store the response
  response = requests.get(
      url,
      headers = {'Accept': 'application/json'}
  )
  response.raise_for_status()
  return response.json()

url = 'https://id.erfgoed.net/afbeeldingen/409414'

try:  
  data = get_url(url)
  # Print the response's JSON data
  print(json.dumps(data, sort_keys=True, indent=4))
except Exception as e:
  print(f"{e}: {e.response.status_code} - {e.response.text}")

#Did I pass the right parameters?

When using a REST collection and passing it parameters to filter or sort on, it's not uncommon for the server to reject certain parameters or certain parameter values. Eg. we can request all images created in a certain year, but what if that year does not exists or will clearly not have any data?

In [1]:
import requests
import json

def search(url, parameters):
  # Make a request and store the response
  response = requests.get(
      url,
      headers = {'Accept': 'application/json'},
      params = parameters
  )
  response.raise_for_status()
  return response.json()

url = 'https://beeldbank.onroerenderfgoed.be/images'
search_parameters = {
  'year': 22000
}

try:  
  data = search(url, search_parameters)
  # Print the response's JSON data
  print(json.dumps(data, sort_keys=True, indent=4))
except Exception as e:
  print(f"{e}: {e.response.status_code} - {e.response.text}")

400 Client Error: Bad Request for url: https://beeldbank.onroerenderfgoed.be/images?year=22000: 400 - {"message": "ValueError('year 22000 is out of range')"}


# Conclusion

By paying attention to the HTTP status codes and writing code expecting things to go wrong, we can write robust scripts that can keep running for hours. Quite often it's not a questing if something will go wrong, but when. Our scripts run on the internet and sometimes requests on the internet fail. There are however paradigms to deal with these errors.

Having learned about [single resources](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/01_get_resource.ipynb), [collections of resources](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/02_get_collection.ipynb) and error handling, you can take a look at some more complicated notebooks that bring multiple topics together:
* [heritage_by_address](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/10_heritage_by_address.ipynb) This notebook explains how you can find all heritage within a certain range of a (Flemish) address.
* [all_getty_skos_matches](https://github.com/OnroerendErfgoed/scriptorium/blob/main/notebooks/11_all_getty_skos_matches.ipynb) This notebook explains how to GET all concepts in the [Agentschap Onroerend Erfgoed Thesaurus](https://thesaurus.onroerenderfgoed.be) that are related to a concept in the [Getty AAT thesaurus](http://vocab.getty.edu/aat).